# 1 Esportazione del dataset

## 1.1 Esportazione da csv

In [1]:
# Controllo se il file CSV esiste
file_path <- "../Dataset/Sentiment_en_tweet_2023.csv"

if (file.exists(file_path)) {
  cat("Il file è stato trovato:", file_path, "\n")
  
  # Provo a leggere il file CSV con il separatore ';'
  df_imported <- try(read.csv(file_path, sep = ";"))
  
  # Verifico se ci sono stati errori nel caricamento
  if (inherits(df_imported, "try-error")) {
    cat("Errore nel caricamento del file. Potrebbero esserci righe malformattate o vuote.\n")
  } else {
    cat("Il file è stato caricato con successo, ma non verrà stampato.\n")
  }
} else {
  cat("Il file non è stato trovato:", file_path, "\n")
}

Il file è stato trovato: ../Dataset/Sentiment_en_tweet_2023.csv 
Il file è stato caricato con successo, ma non verrà stampato.


## 1.2 esportazione da file rda

### 1.2.1 Salvataggio del dataset in file rda

In [4]:
# Percorso del file .rda
rda_file_path <- "../Dataset/Sentiment_en_tweet_2023.rda"

# Salvo il DataFrame df_imported in un file .rda
save(df_imported, file = rda_file_path)

cat("Il file .rda è stato salvato come:", rda_file_path, "\n")


Il file .rda è stato salvato come: ../Dataset/Sentiment_en_tweet_2023.rda 


### 1.2.2 esportazione dal file rdata

In [1]:
# Controllo se il file CSV esiste
file_path <- "../Dataset/Sentiment_en_tweet_2023.rda"

if (file.exists(file_path)) {
  cat("Il file è stato trovato:", file_path, "\n")
  
  # Provo a leggere il file CSV con il separatore ';'
  df_imported <- try(read.csv(file_path, sep = ";"))
  
  # Verifico se ci sono stati errori nel caricamento
  if (inherits(df_imported, "try-error")) {
    cat("Errore nel caricamento del file. Potrebbero esserci righe malformattate o vuote.\n")
  } else {
    cat("Il file è stato caricato con successo, ma non verrà stampato.\n")
  }
} else {
  cat("Il file non è stato trovato:", file_path, "\n")
}

Il file è stato trovato: ../Dataset/Sentiment_en_tweet_2023.rda 


Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"line 3 appears to contain embedded nulls"
Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"line 4 appears to contain embedded nulls"
Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"line 5 appears to contain embedded nulls"
Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
"EOF dentro la stringa virgolettata"
Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
"embedded nul trovati in input"


Il file è stato caricato con successo, ma non verrà stampato.


## 1.3 Ridimensionamento dataset

In [1]:
install.packages("pillar")

package 'pillar' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\smike18\AppData\Local\Temp\RtmpsvfswV\downloaded_packages


In [3]:
library(dplyr)

if(nrow(df_imported) >= 50000) {
  df_sample <- df_imported %>%
    sample_n(size = 50000)
} else {
  stop("Il dataset ha meno di 50.000 righe")
}



Caricamento pacchetto: 'dplyr'


I seguenti oggetti sono mascherati da 'package:stats':

    filter, lag


I seguenti oggetti sono mascherati da 'package:base':

    intersect, setdiff, setequal, union




# 2 - Trattamento di location 

## 2.1 - Eliminazione dei valori null e vuoti dal dataset

In [4]:
# Carico la libreria necessaria
library(dplyr)

# Rimuovo le righe con valori nulli o stringhe vuote nella colonna `location`
df_clean <- df_sample %>%
  filter(!is.na(location) & location != "" & location != "Everywhere" & location != "Mars")

# Visualizzo il numero di istanze tolte
cat("Il dataset da", nrow(df_sample), "istanze è passato a", nrow(df_clean), "istanze.\n")

Il dataset da 50000 istanze è passato a 28416 istanze.


## 2.2 - Uniformare le nazioni

### 2.2.1 Estrazione dei nomi delle nazioni

In [7]:
install.packages("rnaturalearthdata")
install.packages("sf")  # Dipendenze per mappe in formato sf

Warning message:
"il pacchetto 'rnaturalearthdata' è in uso e non sarà installato"
Warning message:
"il pacchetto 'sf' è in uso e non sarà installato"


In [5]:
# Installo il pacchetto se non è già presente
if (!requireNamespace("rnaturalearth", quietly = TRUE)) {
  install.packages("rnaturalearth")
}

# Carico le librerie necessarie
library(sf)
library(rnaturalearth)       # Questa contiene la funzione ne_countries
library(rnaturalearthdata)

# Ottiengo i dati geografici delle nazioni
world <- ne_countries(scale = "medium", returnclass = "sf")

# Estrazione dei nomi delle nazioni
nations <- world$name

# Stampo di alcuni nomi
print(head(nations))

Linking to GEOS 3.9.1, GDAL 3.2.1, PROJ 7.2.1; sf_use_s2() is TRUE


Caricamento pacchetto: 'rnaturalearthdata'


Il seguente oggetto è mascherato da 'package:rnaturalearth':

    countries110




[1] "Zimbabwe"  "Zambia"    "Yemen"     "Vietnam"   "Venezuela" "Vatican"  


### 2.2.2 Caricamento delle librerie necessarie

In [51]:
# Installazione delle librerie necessarie
install.packages(c("tm", "text2vec", "dplyr", "stringdist", "cluster"))


  C'è una versione binaria disponibile, ma la versione con le sorgenti
  è successiva:
           binary source needs_compilation
stringdist 0.9.10 0.9.12              TRUE

  Binaries will be installed
package 'stringdist' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\smike18\AppData\Local\Temp\RtmpSoQXEQ\downloaded_packages


### 2.2.3 Preprocessamento delle stringhe

In [7]:
# Carico le librerie necessarie
library(dplyr)
library(tm)         # Per la manipolazione di testo
library(text2vec)    # Per la creazione di vettori di testo
library(cluster)     # Per il clustering
library(stringdist)  # Per calcolare la similarità tra stringhe

# Preprocesso location
locations_clean <- df_clean %>%
  mutate(location = as.character(location)) %>%  
  mutate(location = tolower(location)) %>%
  mutate(location = removePunctuation(location)) %>%
  mutate(location = removeNumbers(location)) %>%
  mutate(location = stripWhitespace(location))

# Verifico che la colonna `location` sia una colonna di stringhe
print(str(locations_clean$location))

 chr [1:28416] "schweiz" "blue sapphire bloomington" "not of this world" ...
NULL


### 2.2.4 Creazione dataset con nomi delle città e delle nazioni

In [8]:
# Scarico e carico il dataset
download.file("https://download.geonames.org/export/dump/cities15000.zip", destfile = "cities15000.zip")
unzip("cities15000.zip")

# Leggo il file in R
city_data <- read.delim("cities15000.txt", header = FALSE, sep = "\t")

# Seleziono solo le colonne principali (nome città e nazione)
city_country_dict <- city_data %>% select(V2, V5, V6, V9)  # V2 = città, V9 = nazione
colnames(city_country_dict) <- c("city","lat","lon","iso_country")

print(head(city_country_dict))


                city      lat      lon iso_country
1       les Escaldes 42.50729  1.53414          AD
2   Andorra la Vella 42.50779  1.52109          AD
3          WarÄ«sÄ\201n 25.16744 55.40708          AE
4         Umm Suqaym 25.15491 55.21015          AE
5 Umm Al Quwain City 25.56473 55.55517          AE
6    Ar RÄ\201shidÄ«yah 25.22499 55.38947          AE


In [ ]:
# Installo il pacchetto countrycode (se non è già installato)
install.packages("countrycode")

In [9]:
library(countrycode)

# Uso countrycode per associare il codice ISO al nome del paese
nations_from <- countrycode(city_country_dict$iso_country, "iso2c", "country.name")

# Verifico che l'associazione sia corretta
df_nations <- data.frame(city_country_dict, nations_from)

df_nations

Warning message:
"Some values were not matched unambiguously: XK
"


city,lat,lon,iso_country,nations_from
<chr>,<dbl>,<dbl>,<chr>,<chr>
les Escaldes,42.50729,1.53414,AD,Andorra
Andorra la Vella,42.50779,1.52109,AD,Andorra
WarÄ«sÄn,25.16744,55.40708,AE,United Arab Emirates
Umm Suqaym,25.15491,55.21015,AE,United Arab Emirates
Umm Al Quwain City,25.56473,55.55517,AE,United Arab Emirates
Ar RÄshidÄ«yah,25.22499,55.38947,AE,United Arab Emirates
Ras Al Khaimah City,25.78953,55.94320,AE,United Arab Emirates
Zayed City,23.65416,53.70522,AE,United Arab Emirates
Khawr FakkÄn,25.33132,56.34199,AE,United Arab Emirates


#### 2.2.4.1 Aggiunta dizionario dei sinonimi

In [10]:
load("./countrySynonyms.rda")
countrySynonyms

ID,ISO3,name1,name2,name3,name4,name5,name6,name7,name8
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,afg,Afghanistan,Islamic State of Afghanistan,,,,,,NA
2,ala,Aland Islands,,,,,,,NA
3,alb,Albania,Republic of Albania,,,,,,NA
4,dza,Algeria,People's Democratic Republic of Algeria,,,,,,NA
5,asm,American Samoa,Territory of American Samoa,,,,,,NA
6,and,Andorra,Principality of Andorra,,,,,,NA
7,ago,Angola,Republic of Angola,,,,,,NA
8,aia,Anguilla,,,,,,,NA
9,ata,Antarctica,,,,,,,NA


#### 2.2.4.2 Aggiunta di iso3 al dataset delle nazioni

In [11]:
library(countrycode)
df_nations$iso3 <- countrycode(df_nations$iso_country, origin = "iso2c", destination = "iso3c")

df_nations

Warning message:
"Some values were not matched unambiguously: XK
"


city,lat,lon,iso_country,nations_from,iso3
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
les Escaldes,42.50729,1.53414,AD,Andorra,AND
Andorra la Vella,42.50779,1.52109,AD,Andorra,AND
WarÄ«sÄn,25.16744,55.40708,AE,United Arab Emirates,ARE
Umm Suqaym,25.15491,55.21015,AE,United Arab Emirates,ARE
Umm Al Quwain City,25.56473,55.55517,AE,United Arab Emirates,ARE
Ar RÄshidÄ«yah,25.22499,55.38947,AE,United Arab Emirates,ARE
Ras Al Khaimah City,25.78953,55.94320,AE,United Arab Emirates,ARE
Zayed City,23.65416,53.70522,AE,United Arab Emirates,ARE
Khawr FakkÄn,25.33132,56.34199,AE,United Arab Emirates,ARE


#### 2.2.4.3 Associazione dei sinonimi per ogni nazione

In [12]:
library(dplyr)

# Collasso le colonne `name1` a `name8` in una colonna `vettore_stringhe`, escludendo stringhe vuote
countrySynonyms$vettore_stringhe <- apply(countrySynonyms[, 3:9], 1, function(x) {
  # Rimuovi i valori vuoti e restituisci solo quelli non vuoti come vettore
  non_empty_values <- na.omit(x[x != ""])
  return(non_empty_values)  # Ritorna come vettore di stringhe
})

# Seleziono solo la colonna vettore_stringhe e rinominala in sinonimi
countrySynonyms_selected <- countrySynonyms %>%
  rename(iso3 = ISO3) %>%
  select(iso3, vettore_stringhe) %>%
  rename(sinonimi = vettore_stringhe)

# Trasformo solo la colonna iso3 in minuscolo in df_nations
df_nations <- df_nations %>%
  mutate(iso3 = tolower(iso3))

# Eseguo la join tra countrySynonyms_selected e df_nations sulla colonna iso3
df_nations_with_sinonimi <- left_join(df_nations, countrySynonyms_selected, by = "iso3")

# Visualizzo il risultato
print(head(df_nations_with_sinonimi$sinonimi))

Warning message in left_join(df_nations, countrySynonyms_selected, by = "iso3"):
"Detected an unexpected many-to-many relationship between `x` and `y`.
i Row 21208 of `x` matches multiple rows in `y`.
i Row 6 of `y` matches multiple rows in `x`.
i If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence this warning."


[[1]]
                    name1                     name2 
                "Andorra" "Principality of Andorra" 

[[2]]
                    name1                     name2 
                "Andorra" "Principality of Andorra" 

[[3]]
                 name1 
"United Arab Emirates" 

[[4]]
                 name1 
"United Arab Emirates" 

[[5]]
                 name1 
"United Arab Emirates" 

[[6]]
                 name1 
"United Arab Emirates" 



### 2.2.5 Applicazione algoritmo di classificazione

#### 2.2.5.1 Preprocessing del testo

In [13]:
library(dplyr)
library(tm)
library(parallel)

# Funzione di preprocessing del testo
preprocess_text <- function(text) {
  text <- tolower(text)  # Converto tutto in minuscolo
  text <- tm::removePunctuation(text)  # Rimuovo la punteggiatura
  text <- tm::removeNumbers(text)  # Rimuovo i numeri
  text <- tm::removeWords(text, tm::stopwords("en"))  # Rimuovo le stopwords
  text <- tm::stripWhitespace(text)  # Rimuovo gli spazi bianchi extra
  return(text)
}

# Funzione per concatenare sinonimi
concatenate_synonyms <- function(synonyms) {
  # Unisco i sinonimi in una singola stringa separata da spazi
  if (is.list(synonyms)) {
    return(sapply(synonyms, function(x) paste(x, collapse = " ")))
  } else {
    return(synonyms)
  }
}

# Funzione di preprocessing su tutte le colonne testuali in parallelo
parallel_preprocess <- function(df, text_columns) {
  num_cores <- detectCores() - 1  # Usa un core in meno
  cl <- makeCluster(num_cores)
  
  # Carico i pacchetti necessari su ogni nodo del cluster
  clusterEvalQ(cl, {
    library(tm)
  })
  
  # Esporto le funzioni e i dati necessari nel cluster
  clusterExport(cl, list("preprocess_text", "concatenate_synonyms", "df"))
  
  # Applico la concatenazione e il preprocessing su ciascuna colonna di testo
  df[text_columns] <- parLapply(cl, df[text_columns], function(column) {
    concatenated <- concatenate_synonyms(column)  # Concatenazione sinonimi
    sapply(concatenated, preprocess_text)  # Preprocessing
  })
  
  # Chiudo il cluster
  stopCluster(cl)
  return(df)
}

# Definisco le colonne da preprocessare e applica la funzione
text_columns <- setdiff(names(df_nations_with_sinonimi), c("lon", "lat"))  # Esclude 'lon' e 'lat'
df_nations_with_sinonimi <- parallel_preprocess(df_nations_with_sinonimi, text_columns)

# Visualizza il risultato
print(head(df_nations_with_sinonimi))


                city      lat      lon iso_country         nations_from iso3
1       les escaldes 42.50729  1.53414          ad              andorra     
2   andorra la vella 42.50779  1.52109          ad              andorra     
3          warä«sä\201n 25.16744 55.40708          ae united arab emirates     
4         umm suqaym 25.15491 55.21015          ae united arab emirates     
5 umm al quwain city 25.56473 55.55517          ae united arab emirates     
6    ar rä\201shidä«yah 25.22499 55.38947          ae united arab emirates     
                      sinonimi
1 andorra principality andorra
2 andorra principality andorra
3         united arab emirates
4         united arab emirates
5         united arab emirates
6         united arab emirates


#### 2.2.5.2 Associazione di città e nazioni

In [10]:
# install.packages("stringdist")
install.packages("fuzzyjoin")

si installa anche la dipendenza 'geosphere'





  C'è una versione binaria disponibile, ma la versione con le sorgenti
  è successiva:
          binary source needs_compilation
geosphere 1.5-18 1.5-20              TRUE

  Binaries will be installed
package 'geosphere' successfully unpacked and MD5 sums checked
package 'fuzzyjoin' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\smike18\AppData\Local\Temp\Rtmpgtn9Q8\downloaded_packages


#### 2.2.5.3 Funzioni di classificazione delle nazioni

In [9]:
install.packages("parallel")

Warning message:
"il pacchetto 'parallel' è un pacchetto base e non dovrebbe essere aggiornato"


#### Funzione per il calcolo delle distanze tra i sinonimi

In [14]:
calculate_distances <- function(query_string, sinonimi_list) {
  # sinonimi_list è una lista di vettori di sinonimi
  
  # Aggiungo un controllo per evitare errori con dati problematici
  if (length(sinonimi_list) == 0 || !is.list(sinonimi_list)) {
    return(list())  # Restituisce una lista vuota in caso di errore
  }
  
  # Creo una lista per contenere le distanze
  all_distances <- lapply(sinonimi_list, function(sinonimi) {
    # Aggiungo un controllo per evitare errori sui sinonimi
    if (length(sinonimi) == 0 || !is.character(sinonimi)) {
      return(rep(NA, length(sinonimi)))  # Restituisce NA se i sinonimi non sono validi
    }
    
    # Calcolo la distanza per ogni sinonimo nel vettore di sinonimi
    distances <- sapply(sinonimi, function(synonym) {
      dist <- stringdist::stringdist(query_string, synonym, method = "jw")
      return(dist)
    })
    # Restituisco il vettore di distanze per il gruppo di sinonimi
    return(distances)
  })
  
  return(all_distances)
}

##### funzione ottimizzata

In [15]:
calculate_distances <- function(query_string, sinonimi_list) {
    # Controllo se sinonimi_list è una lista valida
    if (!is.list(sinonimi_list) || length(sinonimi_list) == 0) {
        return(numeric(0))
    }

    # Concatenazione e calcolo della distanza con `vapply`
    all_distances <- vapply(sinonimi_list, function(sinonimi) {
        # Controllo sinonimi validi
        if (is.character(sinonimi) && length(sinonimi) > 0) {
            concatenated_sinonimi <- paste(sinonimi, collapse = " ")
            stringdist::stringdist(query_string, concatenated_sinonimi, method = "jw")
        } else {
            NA_real_  # Restituisco NA se i sinonimi non sono validi
        }
    }, FUN.VALUE = numeric(1))  # Specifico che il risultato atteso è numerico

    return(all_distances)
}

#### Funzione per la comparazione delle distanze e il ritorno della nazione

In [16]:
find_country_fast <- function(query_string, dataset, threshold = 0.2) {
    cat("Processing:", query_string, "\n")  # Debug: stampa il valore della query_string
    
    if (is.na(query_string)) {
        return("unknown")
    }
    
    # Calcolo le distanze con `nations_from`
    nation_dists <- stringdist::stringdist(query_string, dataset$nations_from, method = "jw")
    nation_min_dist <- min(nation_dists, na.rm = TRUE)

    if(nation_min_dist <= threshold){
         return(dataset$nations_from[which.min(nation_dists)])
    }
    
    # Calcolo le distanze con `city`
    city_dists <- stringdist::stringdist(query_string, dataset$city, method = "jw")
    city_min_dist <- min(city_dists, na.rm = TRUE)

    if(city_min_dist <= threshold){
         return(dataset$nations_from[which.min(city_dists)])
    }

    # Calcolo le distanze con `sinonimi`
    # sinonimi_list <- dataset$sinonimi  # Ogni elemento è un vettore di sinonimi
    # all_sin_dists <- calculate_distances(query_string, sinonimi_list)

    sinonimi_dists <- stringdist::stringdist(query_string, dataset$sinonimi, method = "jw")
    sinonimi_min_dist <- min(sinonimi_dists, na.rm = TRUE)
    
    # Trovo la distanza minima tra tutti i sinonimi
    # sinonimi_min_dist <- min(sapply(all_sin_dists, min), na.rm = TRUE)

    if(sinonimi_min_dist <= threshold){
         return(dataset$nations_from[which.min(sinonimi_dists)])
    }

    return("unknown")
}

#### Funzione main con parallelizzazione

In [17]:
library(dplyr)
library(stringdist)
library(parallel)

# Carico il dataset delle città e nazioni
dataset <- df_nations_with_sinonimi

# Vettore di stringhe da classificare
input_strings <- locations_clean$location

# Creo un cluster di core per la parallelizzazione
num_cores <- detectCores() - 1
cl <- makeCluster(num_cores)

# Esporto tutte le variabili necessarie nel cluster
clusterExport(cl, list("input_strings", "find_country_fast", "dataset", "calculate_distances", "stringdist"))

# Parallelizzo l'operazione su più core
classified_countries <- parLapply(cl, input_strings, find_country_fast, dataset = dataset)

# Unisco i risultati
classified_countries <- unlist(classified_countries)

# Chiudo il cluster
stopCluster(cl)

# Visualizzo i risultati
print(classified_countries)


    [1] "germany"                     "unknown"                    
    [3] "unknown"                     "syria"                      
    [5] "unknown"                     "spain"                      
    [7] "belgium"                     "poland"                     
    [9] "unknown"                     "ukraine"                    
   [11] "united kingdom"              "united states"              
   [13] "romania"                     "united states"              
   [15] "unknown"                     "unknown"                    
   [17] "canada"                      "unknown"                    
   [19] "unknown"                     "united states"              
   [21] "united states"               "australia"                  
   [23] "united kingdom"              "canada"                     
   [25] "united states"               "unknown"                    
   [27] "canada"                      "united states"              
   [29] "united states"               "ukraine" 

## 2.3 Associazione dei nuovi country e delle coordinate al dataset

### 2.3.1 Mapping country -> coordinate

#### 2.3.1.1 estrazione feature importanti da df_nations 

In [18]:
library(dplyr)

# Creo un nuovo dataset con le colonne richieste e 'nations_from' in minuscolo
df_nations_unique <- df_nations %>%
  select(nations_from, lat, lon, iso3, iso_country) %>%  # Seleziona le colonne
  mutate(nations_from = tolower(nations_from)) %>%  # Converte 'nations_from' in minuscolo
  distinct(nations_from, .keep_all = TRUE)  # Mantieni solo le righe con 'nations_from' unici

# Visualizzo il nuovo dataset
head(df_nations_unique)


,nations_from,lat,lon,iso3,iso_country
,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,andorra,42.50729,1.53414,and,AD
2,united arab emirates,25.16744,55.40708,are,AE
3,afghanistan,30.95962,61.86037,afg,AF
4,antigua & barbuda,17.12096,-61.84329,atg,AG
5,anguilla,18.21704,-63.05783,aia,AI
6,albania,39.87534,20.00477,alb,AL


#### 2.3.1.2 associazione tra i risultati ottenuti e il dataset

In [19]:
library(dplyr)

# Converto 'classified_countries' in un data frame
df_classified_countries <- data.frame(nations_from = classified_countries, stringsAsFactors = FALSE)

# Converte 'nations_from' in minuscolo nel dataset 'df_classified_countries' per uniformità
df_classified_countries <- df_classified_countries %>%
  mutate(nations_from = tolower(nations_from))

# Faccio la join tra 'df_classified_countries' e 'df_nations_unique' sulla colonna 'nations_from'
df_joined <- df_classified_countries %>%
  left_join(df_nations_unique, by = "nations_from")

# Visualizzo il risultato
head(df_joined)


,nations_from,lat,lon,iso3,iso_country
,<chr>,<dbl>,<dbl>,<chr>,<chr>
1,germany,50.72724,12.48839,deu,DE
2,unknown,NA,NA,NA,NA
3,unknown,NA,NA,NA,NA
4,syria,33.96921,36.65729,syr,SY
5,unknown,NA,NA,NA,NA
6,spain,37.11906,-3.58400,esp,ES


### 2.3.2 Rimozione di location e inserimento di df_joined

In [20]:
# Rimuovo la colonna 'location' da df_clean
df_without_location <- locations_clean %>%
  select(-location)

# Aggiungo le colonne da df_joined (senza 'nations_from' se non desideri mantenerla)
df_without_location <- df_without_location %>%
  bind_cols(df_joined)

# Visualizzo il nuovo df_clean
colnames(df_without_location)


[1] "userid"                  "username"               
 [3] "acctdesc"                "following"              
 [5] "followers"               "totaltweets"            
 [7] "usercreatedts"           "tweetid"                
 [9] "tweetcreatedts"          "retweetcount"           
[11] "text"                    "hashtags"               
[13] "language"                "coordinates"            
[15] "favorite_count"          "is_retweet"             
[17] "original_tweet_id"       "original_tweet_userid"  
[19] "original_tweet_username" "in_reply_to_status_id"  
[21] "in_reply_to_user_id"     "in_reply_to_screen_name"
[23] "is_quote_status"         "quoted_status_id"       
[25] "quoted_status_userid"    "quoted_status_username" 
[27] "extractedts"             "sentiment"              
[29] "score"                   "nations_from"           
[31] "lat"                     "lon"                    
[33] "iso3"                    "iso_country"

## 2.4 Esporta il nuovo dataset

In [21]:
# Esporto il dataset df_clean in un file CSV
write.csv(df_without_location, "../Dataset/Sentiment_en_tweet_2023_new_location.csv", row.names = FALSE)
